- Since 01/19/2019
    - Half of all large down spikes (2+ std moves) have occurred before 11 am
    - A quarter of all large up spikes (2+ std moves) have occurred after 3 pm
    - **Avoid trying to defend call spreads after 3 pm**
    - **Avoid trying to defend put spreads before 11 am**

In [5]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import math
def time_block_strtoactual(i):
    curr_time_block = dt.datetime.today().replace(hour = 9, minute = 30, second = 0, microsecond = 0) + dt.timedelta(minutes = 30*(i-1))
    return curr_time_block.strftime("%H:%M")

In [2]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def time_remaining(weekly_options, curr_spx):

    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')

def get_spreads(weekly_options, curr_spx, curr_vix, curr_skew, expiry_index = 0, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()
    
    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']
    
    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)
        
    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    shorts = weekly_options[['Bid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Ask','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-1)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-1)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)

    stepsize = 0.01
    period_iv = np.sqrt(time_remaining)*curr_vix/100
    
    skew_implied_2std = 0.027*(curr_skew - 100)/10
    skew_implied_3std = 0.006*(curr_skew - 100)/10

    period_downside_iv_2std = (-2*period_iv)/norm.ppf(skew_implied_2std,0,1)
    period_downside_iv_3std = (-3*period_iv)/norm.ppf(skew_implied_3std,0,1)

    spreads['EV'] = np.nan
    spreads['Win Prob'] = np.nan

    ev_lsts = []

    for idx, row in spreads.iterrows():

        if typ == 'P':
            ev_df = pd.DataFrame({'SPX': np.arange(row.Long_Strike, row.Short_Strike + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row.Short_Strike + stepsize)]
            ev_df['PnL'] = ev_df['SPX'] - row.Short_Strike + row.Credit
        if typ == 'C':
            ev_df = pd.DataFrame({'SPX': np.arange(row.Short_Strike, row.Long_Strike + stepsize, stepsize)})
            ev_df = ev_df[(ev_df['SPX'] < row.Long_Strike + stepsize)]
            ev_df['PnL'] = row.Short_Strike - ev_df['SPX'] + row.Credit

        ev_df['Prob'] = norm.cdf(ev_df['SPX']/curr_spx - 1,0, period_iv)
        ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - period_iv)]['SPX']/curr_spx - 1,
                                                                       0, period_downside_iv_2std)
        ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['Prob'] = norm.cdf(ev_df[ev_df.SPX < curr_spx*(1 - 2*period_iv)]['SPX']/curr_spx - 1,
                                                                         0, period_downside_iv_3std)
        
        lb = ev_df.loc[0,'Prob']
        ub = ev_df.loc[len(ev_df) - 1,'Prob']
        ev_df['Prob'] = ev_df['Prob'].diff()
        ev_df.loc[0,'Prob'] = lb
        ev_df.loc[len(ev_df) - 1, 'Prob'] = 1 - ub
        ev_df['EV'] = ev_df.Prob*ev_df.PnL

        ev_lsts.append(ev_df)
        total_ev = sum(ev_df['EV'])
        win_prob = sum(ev_df[ev_df['PnL'] >= 0]['Prob'])

        spreads.loc[idx, 'EV'] = total_ev
        spreads.loc[idx, 'Win Prob'] = win_prob
    
    spreads['Expiry'] = expirations_dates[expiry_index]
    spreads.Expiry = pd.to_datetime(spreads['Expiry'])
    return spreads, ev_lsts


#%% Stats
def get_block_stats(curr_block, block_label):

    curr_spx_trend = curr_block.close_spx - curr_block.reset_index(drop = True).loc[0,'close_spx']
    curr_vix_trend = curr_block.close_vix - curr_block.reset_index(drop = True).loc[0,'close_vix']
    spx_X = curr_spx_trend.index ## X usually means our input variables (or independent variables)
    spx_Y = curr_spx_trend ## Y usually means our output/dependent variable

    vix_X = curr_vix_trend.index ## X usually means our input variables (or independent variables)
    vix_Y = curr_vix_trend ## Y usually means our output/dependent variable

    # # Note the difference in argument order
    spx_ols = sm.OLS(spx_Y, spx_X).fit().summary()
    spx_ols_r2 = float(pd.read_html(spx_ols.tables[0].as_html())[0].loc[1,3])
    spx_ols_coeff = float(pd.read_html(spx_ols.tables[1].as_html())[0].loc[1,1])

    vix_ols = sm.OLS(vix_Y, vix_X).fit().summary()
    vix_ols_r2 = float(pd.read_html(vix_ols.tables[0].as_html())[0].loc[1,3])
    vix_ols_coeff = float(pd.read_html(vix_ols.tables[1].as_html())[0].loc[1,1])

    spx_bar_std = np.std(curr_block.close_spx/curr_block.open_spx)
    vix_bar_std = np.std(curr_block.close_vix/curr_block.open_vix)

    spx_block_std = np.std((curr_block.close_spx.shift(1)/curr_block.close_spx).dropna())
    vix_block_std = np.std((curr_block.close_vix.shift(1)/curr_block.close_vix).dropna())

    spx_adf_sig = ts.adfuller(np.log(curr_block.close_spx))[1]
    vix_adf_sig = ts.adfuller(np.log(curr_block.close_vix))[1]

    spx_block_return = curr_block.loc[len(curr_block) - 1, 'close_spx']/curr_block.loc[0, 'close_spx'] - 1
    vix_block_return = curr_block.loc[len(curr_block) - 1, 'close_vix']/curr_block.loc[0, 'close_vix'] - 1
    
    spx_vix_corr = np.corrcoef(curr_block.close_spx,curr_block.close_vix)[0,1]
    
    block_stats = pd.DataFrame({'Block_Label': block_label,
                                'SPX_OLS_Coeff': spx_ols_coeff, 
                                'SPX_OLS_R2': spx_ols_r2,
                                'SPX_bar_std': spx_bar_std,
                                'SPX_block_std': spx_block_std,
                                'SPX_adf_sig': spx_adf_sig,
                                'SPX_block_return': spx_block_return,
                                'VIX_OLS_Coeff': vix_ols_coeff, 
                                'VIX_OLS_R2': vix_ols_r2,
                                'VIX_bar_std': vix_bar_std,
                                'VIX_block_std': vix_block_std,
                                'VIX_adf_sig': vix_adf_sig,
                                'VIX_block_return': vix_block_return,
                                'SPX_VIX_corr': spx_vix_corr,
                                'VIX_open': curr_block.loc[0,'close_vix']}, index = [0])
    return block_stats

def day_block_stats(curr_data):
    block_label = 1
    day_timeblock_stats = []
    for time_block in range(0,len(curr_data),30):
        if time_block < len(curr_data):
            curr_block = curr_data.loc[time_block:time_block + 30, :].reset_index(drop = True)
            day_timeblock_stats.append(get_block_stats(curr_block, "Time{}".format(block_label)))
        block_label += 1

    day_timeblock_stats = pd.concat(day_timeblock_stats, axis = 0).reset_index(drop = True)

    return day_timeblock_stats
#%%


In [43]:
client_id = 12

number_of_expiries = 6


ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

spx = Index('SPX', 'CBOE')
vix = Index('VIX', 'CBOE')
skew = Index('SKEW', 'CBOE')
ib.qualifyContracts(skew)
ib.qualifyContracts(spx)

ib.reqHeadTimeStamp(spx, whatToShow='TRADES', useRTH=True)
ib.reqHeadTimeStamp(vix, whatToShow='TRADES', useRTH=True)

end_date = '' #'20100506 13:00:00 PST'
duration = '1 D'

spx_bars = ib.reqHistoricalData(
        spx,
        endDateTime=end_date,
        durationStr=duration,
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

vix_bars = ib.reqHistoricalData(
        vix,
        endDateTime=end_date,
        durationStr=duration,
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1)

spx_df = util.df(spx_bars).set_index('date')
spx_df.index = pd.to_datetime(spx_df.index)

vix_df = util.df(vix_bars).set_index('date')
vix_df.index = pd.to_datetime(vix_df.index)

curr_spx = spx_df.reset_index().loc[len(spx_df) - 1, 'close']

curr_vix = vix_df.reset_index().loc[len(vix_df) - 1, 'close']

curr_skew = ib.reqMktData(skew, '', False, False).marketPrice()

if math.isnan(curr_skew):
    skew = pd.read_csv('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv', 
                       skiprows = 1)[['Date','SKEW']].set_index('Date')
    curr_skew = skew.reset_index().loc[len(skew) - 1, 'SKEW']

chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

chains_df = util.df(chains)
chain = next(c for c in chains if c.tradingClass == 'SPXW' and c.exchange == 'SMART')

bounds = curr_spx*(curr_vix/(100*np.sqrt(52)))

strikes = [strike for strike in chain.strikes
           if strike % 5 == 0
           and curr_spx - bounds < strike < curr_spx + bounds]
expirations = sorted(exp for exp in chain.expirations)[:number_of_expiries]
expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in expirations]

rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART')
        for right in rights
        for expiration in expirations
        for strike in strikes]

ib.qualifyContracts(*contracts)

spx_options = ib.reqTickers(*contracts)

puts_rows = []
calls_rows = []

for curr_option in spx_options:
    curr_row = create_dfrow(curr_option)
    if curr_option.contract.right == 'P':
        puts_rows.append(curr_row)
    elif curr_option.contract.right == 'C':
        calls_rows.append(curr_row)

puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])

puts = time_remaining(puts, curr_spx)
calls = time_remaining(calls, curr_spx)

curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
puts.to_csv('ib_puts' + curr_time + '.csv')
calls.to_csv('ib_calls' + curr_time + '.csv')


ib.disconnect()

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [44]:
reg_estimates = pd.read_csv('reg_estimates.csv', index_col = 0)

index_intraday = spx_df.join(vix_df, lsuffix = '_spx', rsuffix = '_vix').reset_index()
#index_intraday[['date','close_spx','close_vix']].set_index('date').plot(figsize = (20,10), secondary_y = 'close_vix')
latest_intraday_stats = day_block_stats(index_intraday)
latest_intraday_stats['SPX_trend'] = latest_intraday_stats['SPX_OLS_Coeff']*latest_intraday_stats['SPX_OLS_R2']
latest_intraday_stats['VIX_trend'] = latest_intraday_stats['VIX_OLS_Coeff']*latest_intraday_stats['VIX_OLS_R2']
latest_intraday_stats['SPX_Forecast'] = np.nan

for idx, row in latest_intraday_stats.iterrows():
    curr_estimates = reg_estimates[reg_estimates.Block_Label == row.Block_Label]
    curr_estimates = curr_estimates.join(row).iloc[:,[0,3]]
    curr_estimates.columns = ['Coeff', 'Vals']
    latest_intraday_stats.loc[idx,'SPX_Forecast'] = sum(curr_estimates.Coeff*curr_estimates.Vals)

latest_intraday_stats = latest_intraday_stats.T
latest_intraday_stats.columns = [time_block_strtoactual(i + 1) + " - " + time_block_strtoactual(i + 2) for i in latest_intraday_stats.columns.tolist()]
latest_intraday_stats.iloc[1:,:]


,09:30 - 10:00,10:00 - 10:30,10:30 - 11:00,11:00 - 11:30,11:30 - 12:00,12:00 - 12:30,12:30 - 13:00,13:00 - 13:30,13:30 - 14:00,14:00 - 14:30,14:30 - 15:00,15:00 - 15:30
SPX_OLS_Coeff,0.0170,0.0456,-0.0055,0.0092,0.1036,-0.1581,0.0333,0.2130,0.0404,-0.3159,0.1791,0.0700
SPX_OLS_R2,0.0560,0.4050,-0.0110,0.0840,0.8520,0.8050,0.1930,0.9710,0.4340,0.8450,0.8480,0.7630
SPX_VIX_corr,-0.5805,-0.5564,0.4007,0.0567,-0.9121,-0.9719,-0.8851,-0.9387,-0.6968,-0.9489,-0.8843,-0.7113
SPX_adf_sig,0.1376,0.8907,0.7839,0.0200,0.5236,0.9433,0.0153,0.5656,0.8670,0.3743,0.0743,0.0270
SPX_bar_std,0.0002,0.0002,0.0001,0.0001,0.0002,0.0003,0.0002,0.0001,0.0002,0.0006,0.0003,0.0002
SPX_block_return,0.0003,0.0011,-0.0005,0.0002,0.0005,-0.0024,0.0001,0.0022,0.0007,-0.0016,0.0010,0.0004
SPX_block_std,0.0002,0.0002,0.0001,0.0001,0.0002,0.0003,0.0002,0.0001,0.0002,0.0006,0.0003,0.0002
VIX_OLS_Coeff,-0.0055,-0.0017,-0.0020,-0.0107,-0.0059,0.0106,0.0009,-0.0119,-0.0018,0.0089,-0.0075,-0.0025
VIX_OLS_R2,0.7810,0.4540,0.6380,0.9630,0.8630,0.8690,0.0220,0.9310,0.7280,0.6780,0.8720,0.6980
VIX_adf_sig,0.6380,0.0194,0.7510,0.0127,0.1879,0.8648,0.9001,0.1316,0.7043,0.3950,0.4981,0.4725


In [45]:
expiry_index = 1


put_spreads, evs = get_spreads(puts, curr_spx, curr_vix, curr_skew, expiry_index)
put_spreads.sort_values('EV', ascending = False)

,Short_Strike,Long_Strike,Short_Delta,Delta,Gamma,Theta,Vega,Credit,MaxLoss,BreakEven,EV,Win Prob,Expiry
15,"2,740.0000","2,735.0000",-0.0583,-0.0074,0.0007,-0.0524,0.0247,0.1234,-487.6600,"2,739.8766",-0.1362,0.9390,2019-02-22
14,"2,745.0000","2,740.0000",-0.0770,-0.0139,0.0010,-0.0928,0.0426,0.1234,-487.6600,"2,744.8766",-0.2404,0.9156,2019-02-22
13,"2,750.0000","2,745.0000",-0.1023,-0.0184,0.0012,-0.1012,0.0505,0.2234,-477.6600,"2,749.7766",-0.2738,0.8865,2019-02-22
12,"2,755.0000","2,750.0000",-0.1322,-0.0256,0.0015,-0.1275,0.0625,0.3234,-467.6600,"2,754.6766",-0.3399,0.8510,2019-02-22
11,"2,760.0000","2,755.0000",-0.1713,-0.0313,0.0018,-0.1041,0.0656,0.4234,-457.6600,"2,759.5766",-0.4407,0.8087,2019-02-22
3,"2,800.0000","2,795.0000",-0.7739,-0.0860,-0.0016,0.4041,-0.1100,2.8734,-212.6600,"2,797.1266",-0.4471,0.3404,2019-02-22
10,"2,765.0000","2,760.0000",-0.2210,-0.0428,0.0020,-0.1248,0.0748,0.6234,-437.6600,"2,764.3766",-0.4765,0.7608,2019-02-22
9,"2,770.0000","2,765.0000",-0.2795,-0.0528,0.0022,-0.0897,0.0727,0.8734,-412.6600,"2,769.1266",-0.4955,0.7076,2019-02-22
4,"2,795.0000","2,790.0000",-0.6958,-0.0917,-0.0006,0.3384,-0.0734,2.4734,-252.6600,"2,792.5266",-0.5256,0.3997,2019-02-22
6,"2,785.0000","2,780.0000",-0.5190,-0.0849,0.0012,0.1194,0.0104,1.7734,-322.6600,"2,783.2266",-0.5495,0.5255,2019-02-22


In [46]:
call_spreads, evs = get_spreads(calls, curr_spx, curr_vix, curr_skew, expiry_index)
call_spreads.sort_values('EV', ascending = False)

,Short_Strike,Long_Strike,Short_Delta,Delta,Gamma,Theta,Vega,Credit,MaxLoss,BreakEven,EV,Win Prob,Expiry
6,"2,775.0000","2,780.0000",0.6461,0.0819,-0.0006,0.1590,-0.0453,2.8734,-212.6600,"2,777.8734",-0.1359,0.4029,2019-02-22
7,"2,780.0000","2,785.0000",0.5673,0.0855,0.0002,0.0669,-0.0109,2.4734,-252.6600,"2,782.4734",-0.2025,0.4644,2019-02-22
8,"2,785.0000","2,790.0000",0.4809,0.0840,0.0010,-0.0367,0.0268,2.0734,-292.6600,"2,787.0734",-0.2641,0.5267,2019-02-22
16,"2,825.0000","2,830.0000",0.0262,0.0068,0.0008,-0.0588,0.0280,0.0734,-492.6600,"2,825.0734",-0.3010,0.9130,2019-02-22
9,"2,790.0000","2,795.0000",0.3937,0.0795,0.0015,-0.1537,0.0618,1.6734,-332.6600,"2,791.6734",-0.3303,0.5884,2019-02-22
15,"2,820.0000","2,825.0000",0.0468,0.0130,0.0012,-0.1054,0.0471,0.1734,-482.6600,"2,820.1734",-0.3363,0.8835,2019-02-22
12,"2,805.0000","2,810.0000",0.1679,0.0447,0.0024,-0.2356,0.0978,0.7234,-427.6600,"2,805.7234",-0.3932,0.7584,2019-02-22
14,"2,815.0000","2,820.0000",0.0759,0.0243,0.0018,-0.1902,0.0767,0.2734,-472.6600,"2,815.2734",-0.4042,0.8476,2019-02-22
13,"2,810.0000","2,815.0000",0.1173,0.0347,0.0022,-0.2314,0.0924,0.4734,-452.6600,"2,810.4734",-0.4064,0.8059,2019-02-22
10,"2,795.0000","2,800.0000",0.3085,0.0681,0.0022,-0.1893,0.0847,1.2734,-372.6600,"2,796.2734",-0.4105,0.6480,2019-02-22
